In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")


### Manual tool call

In [41]:
from langchain.tools import tool
from langchain.messages import HumanMessage, ToolMessage, AIMessage
from langchain_ollama.llms import OllamaLLM
from langchain_ollama.chat_models import ChatOllama

#get the model
model = ChatOllama(model="llama3.2", temperature=0)
model.invoke("tell me a joke")

#define the tool
@tool
def add(x:int, y:int) -> int:
    """Function to add 2 numbers"""
    return x+y

messages = [HumanMessage("add 35 + 156")]
#call the model by binding the tool
tool_llm = model.bind_tools(tools=[add])
response = tool_llm.invoke(messages)
print("Tool call: ",response.tool_calls)
# manually call the tool and append the ToolMessage
for tc in response.tool_calls:
    name = tc["name"]
    args = tc['args']
    result = add.invoke(args)
    messages += [response, ToolMessage(content=str(result), tool_call_id=tc['id'])]

final = tool_llm.invoke(messages)
print(final)

Tool call:  [{'name': 'add', 'args': {'x': 35, 'y': 156}, 'id': '21cf2abf-441a-44e2-a0ee-eea771c2802d', 'type': 'tool_call'}]
content='The result of adding 35 and 156 is 191.' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-11-05T18:09:30.217001955Z', 'done': True, 'done_reason': 'stop', 'total_duration': 575612875, 'load_duration': 381125751, 'prompt_eval_count': 93, 'prompt_eval_duration': 26561543, 'eval_count': 14, 'eval_duration': 155347786, 'model_name': 'llama3.2', 'model_provider': 'ollama'} id='lc_run--c40624d0-b07b-4636-81a8-04eea57cc953-0' usage_metadata={'input_tokens': 93, 'output_tokens': 14, 'total_tokens': 107}


In [44]:
#fake ai message mimicking a tool call
ai_msg = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather",
        "args": {"location": "Mumbai"},
        "id": "call_123"
    }]
)

@tool
def get_weather(location: str) -> str:
    """Finds the weather given a location"""
    return "Sunny, 72 degress"

tool_msg = ToolMessage(content="Sunny, 72 degress", tool_call_id="call_123")

messages = [HumanMessage("What is the weather in Mumbai?"), ai_msg, tool_msg]
tool_llm = model.bind_tools(tools=[get_weather])
response = tool_llm.invoke(messages)
print(response.content)

The current weather in Mumbai is sunny, with a temperature of 72 degrees.


### How pipeline works in LC

In [50]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import SystemMessagePromptTemplate
model = ChatOllama(model="llama3.2", temperature=0)
prompt = (SystemMessagePromptTemplate.from_template("You are a helpful assistant") + "Give me a small report about {topic}")
output_parser = StrOutputParser()
model_chain = prompt | model | output_parser

response = model_chain.invoke({"topic": "LLM on edge device"})
# This directly gives out the response content which is in str format
# print(response)

### .bind -> returns a new runnable with altered parameters

In [62]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

prompt = ChatPromptTemplate.from_template("Answer the question concisely: {question}")
model = ChatOllama(model="llama3.2")
model_chain = prompt | model.bind(options={"temperature":0.7}) | output_parser

result = model_chain.invoke({"question": "Who was bob the builder"})
print(result)

Bob the Builder is a fictional character in a children's television series created by Keith Chapman. He is a can-do, optimistic construction worker who solves problems and builds things with his team, featuring talking machines, in the fictional town of Sunflower Valley.


### Getting structured output

In [66]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

model = ChatOllama(model="llama3.2", temperature=0)
class Movie(BaseModel):
    title: str = Field(..., description="title of the movie")
    release: int = Field(..., description="The year of release")
    summary: str = Field(..., description="Short summary of the Movie")
    rating: float = Field(..., description="The rating of the movie")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie review agent and you provide a detailed description of the movie."),
    ("user", "How was the {name} movie?")
])

model_structured = model.with_structured_output(Movie)
model_chain = prompt | model_structured 

response = model_chain.invoke({"name": "Inception"})
print(response)

title='Inception' release=2010 summary="A mind-bending sci-fi action film that delves into the concept of shared dreaming, where a skilled thief is tasked with planting an idea in someone's mind instead of stealing one. The film follows Cobb (Leonardo DiCaprio), a complex and troubled individual who must navigate multiple levels of dreams within dreams to achieve his mission. Alongside his team, including Arthur (Joseph Gordon-Levitt), Ariadne (Ellen Page), Eames (Tom Hardy), and Saito (Ken Watanabe), Cobb embarks on a perilous journey that blurs the lines between reality and fantasy. With its innovative visual effects, intricate plot twists, and thought-provoking themes, Inception is a cinematic masterpiece that will keep you on the edge of your seat." rating=9.5


#### Concept of Runnable in LC
A Runnable is anything that can *run* to produce an output. I can use these functions with *runnables*: invoke(), batch(), stream(), etc.
Types of runnables
- RunnableLambda -> Wraps a function as runnable
- RunnableMap -> Runs multiple runnables in pararell
- RunnableSequence -> same as the `|` operator  
etc.


In [70]:
# Creating runnables
from langchain_core.runnables import RunnableLambda, RunnableMap
add_one = RunnableLambda(lambda x: x+1)
double = RunnableLambda(lambda x: x*2)

pipeline = add_one | double
print("created runnable:", pipeline.invoke(5))

chain = RunnableMap({
    "square": RunnableLambda(lambda x: x*x),
    "cube": RunnableLambda(lambda x: x*x*x)
})

print("With Mapping Lambdas: ", chain.invoke(5))


created runnable: 12
With Mapping Lambdas:  {'square': 25, 'cube': 125}


### Streamimg Responses
- with agent.stream()
- with model.stream() -> after passing the model through the tool call

In [93]:
from langchain.tools import tool
from langchain.messages import HumanMessage
from langchain.agents import create_agent
from langgraph.config import get_stream_writer

@tool
def get_weather(city: str) -> str:
    """ Get the weather for a given city """
    writer = get_stream_writer()
    writer(f"Looking up data for the city: {city}")
    writer(f"Accquired data for the city: {city}")
    return f"It's always sunny in {city}!"

model = ChatOllama(model="llama3.2", temperature=0)
prompt = {"messages":[HumanMessage(content="Can you give me the weather at SF")]}
agent = create_agent(model, tools=[get_weather])

print("Streaming on updates...")
for chunk in agent.stream(prompt, stream_mode="updates"):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")


Streaming on updates...
step: model
content: [{'type': 'tool_call', 'id': 'b3e3bbe4-af56-46bf-87fa-3c20fd65a12f', 'name': 'get_weather', 'args': {'city': 'SF'}}]
step: tools
content: [{'type': 'text', 'text': "It's always sunny in SF!"}]
step: model
content: [{'type': 'text', 'text': 'The current weather conditions in San Francisco are mostly sunny with a high of 68°F (20°C) and a low of 55°F (13°C).'}]
